# 02_estimators.ipynb

Objetivo: ejemplo reproducible para estimar el ATE (estimand primario) usando IPTW, un intento de TMLE (si la librería está disponible) y DML (si `econml` está instalado).

Notas: ejecutar primero `data_prep.py` para descargar los datasets en `data/` (IHDP, LaLonde, Twins).

In [ ]:
# Instalar dependencias si es necesario (ejecutar una vez)
import sys
import subprocess
def pip_install(pack):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', pack])

# Descomenta para instalar (advertencia: tarda y requiere conexión)
# pip_install('pandas')
# pip_install('numpy')
# pip_install('scikit-learn')
# pip_install('statsmodels')
# pip_install('econml')
# pip_install('zepid')

In [ ]:
# Cargar datos (suponemos ./data/ihdp.csv generado por data_prep.py)
import pandas as pd
from pathlib import Path
DATA = Path(__file__).parent / 'data' if '__file__' in globals() else Path('data')
ihdp_path = DATA / 'ihdp.csv'
if not ihdp_path.exists():
    print('No se encontró ihdp.csv en ./data. Ejecuta data_prep.py primero.')
else:
    df = pd.read_csv(ihdp_path)
    print('Loaded IHDP with shape', df.shape)
    display(df.head())

## 1) IPTW (propensity score + pesos estabilizados)

In [ ]:
# IPTW: estimación simple del ATE usando pesos
import numpy as np
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

try:
    T = df['treatment']  # ajustar nombre de columna si distinto
    Y = df['outcome']    # ajustar nombre de columna si distinto
    X = df.drop(columns=['treatment','outcome'])
except Exception as e:
    print('Ajusta nombres de columnas; columnas disponibles:', df.columns.tolist())
    raise

# propensity score
ps_model = LogisticRegression(max_iter=1000)
ps_model.fit(X, T)
ps = ps_model.predict_proba(X)[:,1]

# stabilized weights
p_t = T.mean()
w = np.where(T==1, p_t/ps, (1-p_t)/(1-ps))

# weighted regression of Y on intercept + T to estimate ATE
wls_mod = sm.WLS(Y, sm.add_constant(T), weights=w)
res = wls_mod.fit(cov_type='HC3')
ate_ipw = res.params[1]
se_ipw = res.bse[1]
print(f'IPTW ATE = {ate_ipw:.4f} (SE {se_ipw:.4f})')

## 2) DML (Double Machine Learning) — usa `econml` si está instalado

In [ ]:
try:
    from econml.dml import LinearDML
    from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
    print('econml disponible — corriendo LinearDML')
    model_y = RandomForestRegressor(n_estimators=100, random_state=123)
    model_t = RandomForestClassifier(n_estimators=100, random_state=123)
    dml = LinearDML(model_y=model_y, model_t=model_t, discrete_treatment=True, random_state=123)
    dml.fit(Y, T, X=X)
    ate_dml = dml.ate(X=X)
    print('DML ATE estimate:', ate_dml)
except Exception as e:
    print('econml no disponible o hubo error:', e)
    print('Puedes instalar econml con: pip install econml')

## 3) TMLE (si `zepid` está disponible) — intento básico

In [ ]:
try:
    from zepid import TMLE
    from zepid.causal.utils import propensity_score
    print('zepid disponible — construyendo ejemplo TMLE')
    df_tmle = df.copy()
    # ajustar nombres columna: treatment -> 'treatment', outcome -> 'outcome'
    tmle = TMLE(df_tmle, exposure='treatment', outcome='outcome')
    # especificar modelos (simple logistic/linear)
    tmle.exposure_model(' + '.join(X.columns), print_results=False)
    tmle.outcome_model('treatment + ' + ' + '.join(X.columns), print_results=False)
    tmle.fit()
    print('TMLE estimate:', tmle.estimate)
except Exception as e:
    print('zepid TMLE no disponible o error:', e)
    print('Instala zepid: pip install zepid')

## Diagnostics y siguientes pasos rápidos
- Revisar balance pre/post IPTW (ASMD) y distribución de pesos (plot).
- Ejecutar simulaciones (01_simulations.ipynb) para escenarios A/B/C y comparar estimadores.
- Guardar resultados en `results/` y generar tablas para el manuscrito.